#  HUNDIR LA FLOTA
    > Desarrollado por Team Minerva

Aquí describimos la estructura y organización del código del juego "Hundir la Flota", que está dividido en cuatro ficheros principales. Cada fichero tiene responsabilidades
específicas que permiten mantener la modularidad y facilitar la modificación y mantenimiento del código.


## 1.  **variables.py**

El fichero variables.py contiene todas las constantes y configuraciones globales necesarias
para el juego. Esto incluye el tamaño del tablero, los tipos de celdas (agua, barco, impacto) y
la estructura de los barcos. La separación de estas configuraciones permite modificar el
juego sin afectar el resto del código, es decir que le da escalabilidad.

<span>![variables.py](imagen_presentacion_1.png)</span>

### Contenido de variables.py

In [24]:
BOARD_SIZE = 10

- Define el tamaño del tablero del juego como una cuadrícula de 10x10.

In [25]:
SHIPS = {
    "Barco 1": 1,  # 4 barcos de 1 posición
    "Barco 2": 2,  # 3 barcos de 2 posiciones
    "Barco 3": 3,  # 2 barcos de 3 posiciones
    "Barco 4": 4   # 1 barco de 4 posiciones
}

- Define los tipos de barcos y cuántos de cada tipo estarán en el juego.
- Además, eesta esttructura nos facilita añadir o quitar barcos sin tocar la lógica principal.

In [26]:
WATER = "0"  # Agua
SHIP = "1"   # Barco
HIT = "X"    # Impacto
MISS = "M"   # Agua disparada


- Establece los valores posibles para las celdas del tablero. Cada valor tiene un significado específico: 

    + WATER ("0"): Indica que esa celda está vacía.
    + SHIP ("1"): Representa una celda ocupada por un barco.
    + HIT ("X"): Marca un disparo que impactó en un barco.
    + MISS ("M"): Indica un disparo que no alcanzó un barco.

## 2.  **clases.py**

Este fichero define la clase Board, que encapsula la lógica relacionada con los tableros del juego, incluyendo la colocación de barcos, los disparos y la visualización. Es uno de los componentes más importantes del código porque gestiona las reglas fundamentales del juego.

### Contenido de clases.py

In [27]:
import numpy as np
from random import randint
#from variables import SHIPS, BOARD_SIZE, WATER, SHIP, HIT, MISS


- Primero importamos las librerias y las variables que nos hemos creado: 

    + numpy: Para crear y manipular matrices como hemos aprendido en los springs.
    + randint: Para manejar numeros enteros aleatorios.
    + variables: Nos servirán para dar forma al tablero y sus métodos.
    

In [28]:
class Board:
    def __init__(self, player_id):
        self.player_id = player_id
        self.size = BOARD_SIZE
        self.grid = np.full((self.size, self.size), WATER)  # Tablero visible
        self.ship_grid = np.full((self.size, self.size), WATER)  # Tablero oculto con los barcos
        self.ships = []  # Lista de coordenadas de barcos

        # Coloca los barcos en posiciones fijas o aleatorias
        self.place_ships()


- Aquí configuramos el "molde" principal del juego, es decir, el tablero, que es donde ocurrirá todo.

- En esta clase llamada "board" podemos ver que hemos definido los atributos:

    +  player_id: Identifica si el tablero pertenece al jugador o a la IA.
    + grid: Representa el tablero visible para el jugador. Que está inicialmente lleno de WATER ("0").
    + ship_grid: Tablero oculto que contiene la posición de los barcos (SHIP).
    + ships: Una lista de coordenadas que almacena la ubicación exacta de cada barco.   

In [29]:
def place_ships(self):
    for ship_name, length in SHIPS.items():
        for _ in range(SHIPS[ship_name]):
            placed = False
            while not placed:
                placed = self.place_single_ship(length)


- Esta función nos permite colocar los barcos sin que se superpongan:

    + Itera sobre el diccionario SHIPS para obtener el nombre y la longitud de cada tipo de barco.
    + Intenta colocar cada barco utilizando el método place_single_ship.
    + Si no puede colocarlo (por ejemplo, porque las posiciones están ocupadas), lo intenta nuevamente hasta que se coloque.

In [30]:
def place_single_ship(self, length):
    direction = randint(0, 3)  # 0 = Norte, 1 = Sur, 2 = Este, 3 = Oeste
    x = randint(0, self.size - 1)
    y = randint(0, self.size - 1)

    coordinates = []
    if direction == 0:  # Norte
        if x - length >= 0:
            coordinates = [(x - i, y) for i in range(length)]
    elif direction == 1:  # Sur
        if x + length < self.size:
            coordinates = [(x + i, y) for i in range(length)]
    elif direction == 2:  # Este
        if y + length < self.size:
            coordinates = [(x, y + i) for i in range(length)]
    else:  # Oeste
        if y - length >= 0:
            coordinates = [(x, y - i) for i in range(length)]

    if all(self.ship_grid[coord[0], coord[1]] == WATER for coord in coordinates):
        for coord in coordinates:
            self.ship_grid[coord[0], coord[1]] = SHIP
        self.ships.append(coordinates)
        return True
    return False


- Aquí es donde creamos la lógica que nos permitirá colocar barco a barco en una posicion aleatoria del tablero. Para ello, creamos una cardinalidad, coloca el barco y nos devuelve un booleano que será usado en place_ships para verificar si esa posición estaba ocupada.

In [31]:
def shoot(self, x, y):
    if self.grid[x, y] == HIT or self.grid[x, y] == MISS:
        return "Ya has disparado en esta coordenada. Intenta con otra."
    if self.ship_grid[x, y] == SHIP:
        self.grid[x, y] = HIT
        return "Disparo certero"
    else:
        self.grid[x, y] = MISS
        return "Disparo fallido"


- Aquí definimos la lógica de los disparos:

    + Si el jugador ya disparó en esa coordenada (HIT o MISS), devuelve un mensaje de error.
    + Si hay un barco (SHIP) en la celda, marca un impacto (HIT) y devuelve "Disparo certero".
    + Si no hay barco, marca un fallo (MISS) y devuelve "Disparo fallido".

In [32]:
def display(self):
    columnas = "   " + " ".join(f"{i:2}" for i in range(self.size))
    print(columnas)
    print("   " + "-" * (self.size * 3))
    for i, fila in enumerate(self.grid):
        fila_completa = " ".join(f"{cell:2}" for cell in fila)
        print(f"{i:2} | {fila_completa}")


Esta función configura la visualización, mostrando el tablero visible (sin barcos) en un formato limpio con coordenadas numeradas.


In [33]:
def display_full_board(self):
    full_board = np.where(self.ship_grid == SHIP, SHIP, self.grid)
    columnas = "   " + " ".join(f"{i:2}" for i in range(self.size))
    print(columnas)
    print("   " + "-" * (self.size * 3))
    for i, fila in enumerate(full_board):
        fila_completa = " ".join(f"{cell:2}" for cell in fila)
        print(f"{i:2} | {fila_completa}")


- Muestra un tablero combinado que revela la posición de los barcos.

In [34]:
def is_sunk(self, ship_coordinates):
    return all(self.grid[x, y] == HIT for x, y in ship_coordinates)

- Determina si un barco específico (identificado por sus coordenadas) ha sido completamente hundido.

<span>![variables.py](imagen_presentacion_3.png)</span>

## 3.  **funciones.py**

El fichero funciones.py contiene funciones auxiliares que facilitan la interacción entre el
jugador y el programa. Estas funciones son la entrada de usuario, el disparo de la
inteligencia artificial y la verificación del estado del juego, con acciones específicas
como la entrada/salida y la lógica del turno.

### Contenido de funciones.py

In [16]:
#from random import randint
#from clases import Board
#from variables import BOARD_SIZE

- Como antes, importamos numeros aleatórios y ahora las clases y variables qu hemos construido.

In [35]:
def print_welcome_message():
    print("¡Bienvenido a Hundir la Flota!")
    print("Este es un juego entre dos jugadores: tú y la máquina.")
    print("El objetivo es hundir todos los barcos del adversario antes que él haga lo mismo contigo.")
    print("Durante tu turno, puedes disparar a coordenadas (x, y), ver los tableros ('Mostrar') o salir del juego ('Salir').")


- Mostrar un mensaje introductorio que explica las reglas del juego al jugador.

In [18]:
def get_player_input():
    while True:
        try:
            user_input = input("Ingresa las coordenadas (x, y) para disparar o 'mostrar' para ver los tableros, 'salir' para salir: ").strip().lower()
            if user_input == "salir":
                return "salir"
            elif user_input == "mostrar":
                return "mostrar"
            else:
                x, y = map(int, user_input.split(','))
                if 0 <= x < 10 and 0 <= y < 10:
                    return x, y
                else:
                    print("Coordenadas fuera de rango. Deben ser entre 0 y 9.")
        except ValueError:
            print("Entrada inválida. Debes ingresar las coordenadas en el formato x,y (por ejemplo, 1,2).")


- Capturar y validar la entrada del jugador para asegurarse de que sea válida antes de procesarla:

    + Solicita al jugador que ingrese una acción o coordenadas.
    + Comprueba si el jugador quiere:
        - Salir del juego ("salir"): Retorna "salir".
        - Mostrar los tableros ("mostrar"): Retorna "mostrar". 
    + Si se ingresan coordenadas (x,y):
        - Intenta convertirlas en números enteros (int).
        - Verifica que estén dentro del rango permitido (0-9).
        - Si son válidas, las retorna como una tupla (x, y). 
    + Si la entrada es inválida, muestra un mensaje de error y pide la entrada nuevamente.


In [19]:
def ai_shoot():
    x, y = randint(0, 9), randint(0, 9)
    return x, y


- Generar disparos aleatorios para la inteligencia artificial (IA):

    + Usa randint para generar dos números aleatorios (x y y) entre 0 y 9.
    + Retorna las coordenadas generadas como una tupla (x, y).


In [20]:
def check_game_over(player):
    for ship in player.ships:
        if not player.is_sunk(ship):
            return False
    return True


- Verificar si todos los barcos de un jugador han sido hundidos, determinando así si el juego ha terminado:

    + Itera sobre la lista de barcos (player.ships) de un jugador.
    + Para cada barco, llama al método is_sunk de la clase Board:
        - Comprueba si todas las coordenadas del barco están marcadas como impactos (HIT).
    + Si al menos un barco no está hundido, retorna False.
    + Si todos los barcos están hundidos, retorna True.

<span>![variables.py](imagen_presentacion_4.png)</span>

## 4.  **main.py**

Este archivo define la función principal del programa, main, que coordina el flujo general del juego. Es el núcleo del juego y combina las funcionalidades de otros archivos para que todo funcione como un sistema cohesivo.

<span>![variables.py](imagen_presentacion_2.png)</span>

### Contenido de main.py

In [21]:
#from funciones import print_welcome_message, get_player_input, ai_shoot, check_game_over
#from clases import Board

- Importamos las clases y las funciones.

In [39]:
def main ():
    print_welcome_message()
    player_board = Board(player_id="Player")
    ai_board = Board(player_id="AI")
    while True:
        # Turno del jugador humanoo
        print("\nTu tablero completo con la ubicación de tus barcos:")
        player_board.display_full_board()
        print("Tablero de la Skynet con tus impactos: ")
        ai_board.display()

        player_action = get_player_input()

        if player_action == "salir":
            print("¡Gracias por jugar! Hasta la próxima.")
            break
        elif player_action == "mostrar":
            continue
        else:
            x, y = player_action
            result = ai_board.shoot(x, y)
            print(f"Disparo en ({x}, {y}): {result.capitalize()}")

            if check_game_over(ai_board):
                print("¡Ganaste! La Skynet ha sido derrotada.")
                break

            # Turno de la Skynet
            x, y = ai_shoot()
            print(f"Skynet dispara en ({x}, {y})")
            result = player_board.shoot(x, y)
            print(f"Resultado: {result.capitalize()}")

            if check_game_over(player_board):
                print("Perdiste. La Skynet ha ganado.")
                break


- Saludamos.
- Inicializamos los tableros.
- El juego se desarrolla en un bucle infinito que solo termina cuando:
    + El jugador elige salir ("salir").
    + Uno de los jugadores gana.  
- Renderizamos el tablero.
- Llama a get_player_input para capturar la acción del jugador:
    + "salir": Termina el juego.
    + "mostrar": Muestra los tableros nuevamente sin realizar una acción.
    + (x, y): Dispara a las coordenadas especificadas.
- Si el jugador dispara a (x, y):
    + Llama al método shoot de ai_board para registrar el disparo.
    + Muestra el resultado ("Disparo certero" o "Disparo fallido").
- Llama a check_game_over para verificar si el jugador ha hundido todos los barcos de la IA.
- Si la IA no tiene más barcos, el jugador gana y el juego termina.
- La IA genera disparos aleatorios usando ai_shoot.
- El disparo se registra en el tablero del jugador humano (player_board.shoot).
- El resultado del disparo se imprime.
- Verifica si la IA ha hundido todos los barcos del jugador.
- Si no quedan barcos, el jugador pierde y el juego termina.
- Permite al jugador salir del juego en cualquier momento escribiendo "salir".
- Muestra un mensaje de despedida y termina el programa.



